Lecture 2 (practice)
======================


## About

For this part of the assignment, you will gain hands-on experience working with Convolutional Neural Networks (CNNs).

* **Custom CNN Model**:

    Design and implement a custom CNN architecture tailored for the CIFAR-10, DTD, and COCO-O datasets. Train your model and evaluate its performance on each dataset. Consider discussing the design choices you made, such as the number of layers, types of layers, activation functions, etc., and how they contribute to your model's performance.

* **Linear Probe of ResNet**:

    Conduct a linear probe using the pre-trained ResNet features on the CIFAR-10, DTD, and COCO-O datasets. Analyze the effectiveness of transfer learning in speeding up training and improving accuracy. You may also want to compare this approach to training a model from scratch.

* **Fine-tuning ResNet model**:

    Fine-tune the ResNet model on the CIFAR-10, DTD, and COCO-O datasets to achieve better performance compared to the linear probe method. Discuss how fine-tuning impacts model accuracy and convergence speed. It's also worth exploring the balance between maintaining pre-trained knowledge and adapting to new tasks.


<hr> 

* The <b><font color="red">red</font></b> color indicates the task that should be done, like <b><font color="red">[TODO]</font></b>: ...
* Addicitional comments, hints are in <b><font color="blue">blue</font></b>. For example <b><font color="blue">[HINT]</font></b>: ...

## Prelimiaries

In [1]:
!pip install datasets
!pip install fiftyone
# !pip install scikit-learn
# !pip install tensorboard jupyter-tensorboard
# !pip install tqdm

In [51]:
import os
import gdown
import zipfile
from tqdm import tqdm
from copy import deepcopy

import datetime
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

import torchvision
import torchvision.transforms as transforms

from datasets import load_dataset # HuggingFace dedicated lib
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

import fiftyone as fo

In [52]:
# make plots a bit nicer
plt.matplotlib.rcParams.update({'font.size': 18, 'font.family': 'serif'})

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


## Auxilary functions

### Training related

In [5]:
def train_and_validate(
    model: nn.Module,    
    train_loader: torch.utils.data.DataLoader,
    val_loader: torch.utils.data.DataLoader,
    num_epoch: int,
    criterion: nn.Module,
    optimizer: optim.Optimizer,
    device: torch.device,
    max_iter: int | None = None,     
    writer: SummaryWriter | None = None
) -> nn.Module:
    """Simple training script."""    
    
    # Model to device
    model.to(device)
    
    # Log interval
    log_interval = max(1, int(len(train_loader) / 100))
    log_counter = 0

    best_val_accuracy = 0.0
    best_model_state = None
    for epoch in range(num_epoch):

        # Training part
        model.train()
        running_loss = 0.0
        running_correct = 0
        running_amount = 0
        for batch_idx, (inputs, labels) in tqdm(enumerate(train_loader), 'training', total=len(train_loader)):
            
            # Move the inputs and labels to the device
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            output = model(inputs)
            loss = criterion(output, labels)
            
            # Backward pass and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Update running loss and accuracy
            _, preds = torch.max(output, 1)
            running_amount += inputs.size(0)
            running_loss += loss.item() * inputs.size(0)
            running_correct += torch.sum(preds == labels.data)

            # Log to tensorboard
            if writer and batch_idx % log_interval == 0:
                running_accuracy = (running_correct.float() / running_amount).item()
                writer.add_scalar('Train/Loss', running_loss / running_amount, log_counter)
                writer.add_scalar('Train/Accuracy', running_accuracy, log_counter)
                log_counter += 1

            if max_iter and batch_idx > max_iter:
                break
                    
        # The train loss and accuracy
        train_loss = running_loss / len(train_loader.dataset)
        train_acc = running_correct.float() / len(train_loader.dataset)

        # Evaluation part
        model.eval()
        running_loss = 0.0
        running_correct = 0
        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, 'evaluation', total=len(val_loader)):
                
                # Move the inputs and labels to the device
                inputs, labels = inputs.to(device), labels.to(device)

                # Forward pass
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                # Update the running loss and accuracy
                running_loss += loss.item() * inputs.size(0)
                running_correct += torch.sum(preds == labels.data)

        # Calculate the validation loss and accuracy
        val_loss = running_loss / len(val_loader.dataset)
        val_acc = running_correct.float() / len(val_loader.dataset)

        # Best model
        if val_acc > best_val_accuracy:
            best_model_state = deepcopy(model.state_dict())
            best_val_accuracy = val_acc

        # Print the epoch results
        print(
            f'Epoch [{epoch + 1}/{num_epoch}], '
            f'train loss: {train_loss:.4f}, train acc: {train_acc:.4f}, '
            f'val loss: {val_loss:.4f}, val acc: {val_acc:.4f}'
        )
        # Log to tensorboard
        if writer:
            writer.add_scalar('TrainEpoch/Loss', train_loss, epoch + 1)
            writer.add_scalar('TrainEpoch/Accuracy', train_acc, epoch + 1)            
            writer.add_scalar('ValidationEpoch/Loss', val_loss, epoch + 1)
            writer.add_scalar('ValidationEpoch/Accuracy', val_acc, epoch + 1)
    
    # Load the best model state before returning
    if best_model_state is not None:
        model.load_state_dict(best_model_state)

    return model

In [6]:
def predict(
    model: nn.Module,    
    data_loader: torch.utils.data.DataLoader,
    device: torch.device,
) -> np.ndarray:
    """ Predict on a given dataloader. """
    model.to(device)
    model.eval()
    predictations = []
    ground_truth_labels = []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            predictations.extend(preds.cpu().numpy())
            ground_truth_labels.extend(labels)
    return np.array(predictations), np.array(ground_truth_labels)

### Models related

In [7]:
class SimpleNet(nn.Module):
    def __init__(self, n_classes: int):
        super(SimpleNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, n_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

### Dataloader related

In [8]:
class DTDdataset(torch.utils.data.Dataset):
    def __init__(self, hf_dataset, transform=None):
        self.dataset = hf_dataset
        self.transform = transform

    def __getitem__(self, idx):
        item = self.dataset[idx]
        image = item['image']
        label = item['label']
        
        if self.transform:
            image = self.transform(image)

        return image, label

    def __len__(self):
        return len(self.dataset)

In [9]:
transform_cifar = transforms.Compose([
    transforms.Resize(32),
    transforms.CenterCrop(32),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

In [10]:
transform_imagenet = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

## Load data

In [11]:
# Set the local folder with the data
path_data = "./data"
os.makedirs(path_data, exist_ok=True)

In [12]:
# Load cifar10 dataset
cifar10_dataset_train = torchvision.datasets.CIFAR10(root=path_data, train=True, download=True, transform=None)
cifar10_dataset_test = torchvision.datasets.CIFAR10(root=path_data, train=False, download=True, transform=None)
cifar10_classes_list = cifar10_dataset_train.classes

Files already downloaded and verified
Files already downloaded and verified


In [13]:
# Load DTD dataset
dtd_dataset = load_dataset("tanganke/dtd", cache_dir=path_data)
dtd_classes_list = dtd_dataset['train'].features['label'].names

In [14]:
# Prepare COCO-O dataset
if not os.path.exists(os.path.join(path_data, 'ood_coco')):
    url = 'https://drive.google.com/uc?id=1aBfIJN0zo_i80Hv4p7Ch7M8pRzO37qbq'
    zip_file_path = os.path.join(path_data, 'ood_coco.zip')
    gdown.download(url, zip_file_path, quiet=False)
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(path_data)
cocoo_classes_list = os.listdir(os.path.join(path_data, 'ood_coco'))

## (optional) Tensorboard

This part is completely optional. If you find it valuable, you may use TensorBoard or other similar tools to track the training process.

In [15]:
# Path for logs
current_time = str(datetime.datetime.now().timestamp())
path_log = 'logs/tensorboard/' + current_time

# Set-up writer
writer = SummaryWriter(path_log)

In [16]:
#%tensorboard --logdir logs/tensorboard

## Training (from scratch)

In [17]:
# Set the device 
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

In [18]:
# We will use 'crossentropy' for all classification problems
criterion = nn.CrossEntropyLoss()

### cifar10

In [19]:
# settings
num_epoch = 3
learning_rate = 0.01
momentum = 0.9
batch_size = 32

In [20]:
# Prepare dataloaders
trainset = torchvision.datasets.CIFAR10(root=path_data, train=True, transform=transform_cifar)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
testset = torchvision.datasets.CIFAR10(root=path_data, train=False, transform=transform_cifar)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

In [21]:
# Let's have a look at the sizes of the loaders
print(len(trainloader))
print(len(testloader))

# Let's check sizes of the batch and their types
images, labels = next(iter(testloader))
print(images.shape, type(images))
print(labels.shape, type(labels))

1563
313
torch.Size([32, 3, 32, 32]) <class 'torch.Tensor'>
torch.Size([32]) <class 'torch.Tensor'>


In [22]:
# Set-up model and optimizer
model_cifar = SimpleNet(n_classes=10)
optimizer = torch.optim.SGD(model_cifar.parameters(), lr=learning_rate, momentum=momentum)

In [23]:
# Let's train!

In [30]:
%%time
best_model = train_and_validate(
    model=model_cifar, 
    train_loader=trainloader, 
    val_loader=testloader, 
    num_epoch=num_epoch, 
    criterion=criterion, 
    optimizer=optimizer, 
    device=device, 
    writer=writer
)

evaluation: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 313/313 [00:01<00:00, 252.41it/s]


Epoch [1/5], train loss: 0.9430, train acc: 0.6709, val loss: 1.2537, val acc: 0.5882


evaluation: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 313/313 [00:01<00:00, 255.35it/s]


Epoch [2/5], train loss: 0.9467, train acc: 0.6721, val loss: 1.2861, val acc: 0.5771


evaluation: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 313/313 [00:01<00:00, 254.32it/s]


Epoch [3/5], train loss: 0.9153, train acc: 0.6816, val loss: 1.3155, val acc: 0.5858


evaluation: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 313/313 [00:01<00:00, 250.58it/s]


Epoch [4/5], train loss: 0.9139, train acc: 0.6799, val loss: 1.2882, val acc: 0.5770


evaluation: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 313/313 [00:01<00:00, 255.49it/s]


Epoch [5/5], train loss: 0.9064, train acc: 0.6868, val loss: 1.2758, val acc: 0.5983
CPU times: user 52.1 s, sys: 6 s, total: 58.1 s
Wall time: 52.6 s


In [31]:
# Make predictations
predictations, true_labels = predict(model=best_model, data_loader=testloader, device=device)

In [32]:
# Detailed analysis (report)
print(classification_report(true_labels, predictations, target_names=cifar10_classes_list))

              precision    recall  f1-score   support

    airplane       0.58      0.66      0.62      1000
  automobile       0.76      0.74      0.75      1000
        bird       0.48      0.52      0.50      1000
         cat       0.36      0.52      0.43      1000
        deer       0.54      0.51      0.52      1000
         dog       0.55      0.44      0.49      1000
        frog       0.76      0.64      0.69      1000
       horse       0.73      0.56      0.63      1000
        ship       0.66      0.75      0.70      1000
       truck       0.75      0.63      0.68      1000

    accuracy                           0.60     10000
   macro avg       0.62      0.60      0.60     10000
weighted avg       0.62      0.60      0.60     10000



In [33]:
# Detailed analysis (confusion matrix)

cm = confusion_matrix(true_labels, predictations)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=cifar10_classes_list)

fig, ax = plt.subplots(figsize=(10, 8))
disp.plot(cmap='Blues', ax=ax, xticks_rotation=90);

array([[661,  31,  49,  26,  25,  18,   7,  15, 137,  31],
       [ 47, 736,   7,  22,  11,   3,   6,  10,  94,  64],
       [ 99,   7, 523, 112,  93,  49,  44,  26,  31,  16],
       [ 40,  10,  82, 524,  67, 144,  58,  31,  22,  22],
       [ 37,   6, 143, 125, 512,  40,  53,  64,  14,   6],
       [ 27,   5,  90, 306,  63, 442,  10,  35,   7,  15],
       [ 15,   7,  80, 159,  62,  16, 640,   4,   6,  11],
       [ 31,  10,  83,  89, 101,  88,  12, 561,   7,  18],
       [114,  25,  13,  41,  11,   5,   5,   2, 751,  33],
       [ 65, 132,  21,  40,  10,   5,   9,  21,  64, 633]])

### DTD

In [34]:
# settings
num_epoch = 5
learning_rate = 0.01
momentum = 0.9
batch_size = 32

In [35]:
# Prepare dataloaders
trainset = DTDdataset(dtd_dataset['train'], transform=transform_cifar)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
testset = DTDdataset(dtd_dataset['test'], transform=transform_cifar)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

<b><font color="red">[TODO]</font></b>: Following the example of 'cifar10' dataset complete the training from scratch for DTD dataset from this point.

What is the accuracy? Why?


In [36]:
# YOUR CODE HERE

### COCO-O

In [37]:
# settings
num_epoch = 5
learning_rate = 0.01
momentum = 0.9
batch_size = 32

<b><font color="red">[TODO]</font></b>: Following the example of 'cifar10' dataset complete the training from scratch for COCO-O dataset from this point.

<b><font color="blue">[HINT]</font></b>: You will need to write the custom class for the dataloader similar to 'DTDdataset'.

What is the accuracy? Why?

In [38]:
# YOUR CODE HERE

## Training (Fine-tuning)

In [39]:
from torchvision.models import resnet18, resnet50
from torchvision.models import ResNet18_Weights, ResNet50_Weights 

### cifar10

In [40]:
# Settings
num_epoch = 3
learning_rate = 0.01
momentum = 0.9
batch_size = 32

In [41]:
# Prepare dataloaders
# Note that we've changed the 'transform'
trainset = torchvision.datasets.CIFAR10(root=path_data, train=True, transform=transform_imagenet)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
testset = torchvision.datasets.CIFAR10(root=path_data, train=False, transform=transform_imagenet)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

In [42]:
# Prepare model
model_cifar = resnet18(weights=ResNet18_Weights.DEFAULT)

In [43]:
# STEP 1: We will first train only the last layer, freezing the rest of parameters.
# This can be considered as a getting 'resnet' features and making linear probe on top of this.
for param in model_cifar.parameters():
    param.requires_grad = False

In [44]:
# Modify the very last layer
num_classes = len(cifar10_classes_list)
model_cifar.fc = torch.nn.Linear(model_cifar.fc.in_features, num_classes)

In [45]:
# Set-up optimizer
optimizer = torch.optim.SGD(model_cifar.parameters(), lr=learning_rate, momentum=momentum)

In [46]:
# We repeat the same training process as before.
# Note: To expedite the process, the parameter max_iter is set to 100. Consider increasing this value if you want to achieve better results.
best_model = train_and_validate(
    model=model_cifar, 
    train_loader=trainloader, 
    val_loader=testloader, 
    num_epoch=num_epoch, 
    criterion=criterion, 
    optimizer=optimizer, 
    device=device, 
    max_iter=100
)

evaluation: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 313/313 [00:19<00:00, 16.14it/s]


Epoch [1/5], train loss: 0.0850, train acc: 0.0353, val loss: 0.8752, val acc: 0.6940


evaluation: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 313/313 [00:19<00:00, 16.40it/s]


Epoch [2/5], train loss: 0.0655, train acc: 0.0435, val loss: 0.8812, val acc: 0.6992


evaluation: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 313/313 [00:19<00:00, 16.27it/s]


Epoch [3/5], train loss: 0.0640, train acc: 0.0444, val loss: 0.9309, val acc: 0.6952


evaluation: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 313/313 [00:19<00:00, 16.25it/s]


Epoch [4/5], train loss: 0.0603, train acc: 0.0455, val loss: 0.8746, val acc: 0.7181


evaluation: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 313/313 [00:19<00:00, 16.07it/s]

Epoch [5/5], train loss: 0.0589, train acc: 0.0456, val loss: 0.7219, val acc: 0.7610


In [47]:
# As the same type of analysis of predictations
predictations, true_labels = predict(model=best_model, data_loader=testloader, device=device)
print(classification_report(true_labels, predictations, target_names=cifar10_classes_list))

              precision    recall  f1-score   support

    airplane       0.73      0.82      0.77      1000
  automobile       0.83      0.88      0.86      1000
        bird       0.75      0.65      0.69      1000
         cat       0.63      0.65      0.64      1000
        deer       0.72      0.72      0.72      1000
         dog       0.72      0.75      0.73      1000
        frog       0.81      0.76      0.79      1000
       horse       0.71      0.83      0.77      1000
        ship       0.84      0.77      0.81      1000
       truck       0.91      0.78      0.84      1000

    accuracy                           0.76     10000
   macro avg       0.77      0.76      0.76     10000
weighted avg       0.77      0.76      0.76     10000



In [48]:
# STEP 2: Relax 'freezing' contstrains and let's train the whole network
for param in model_cifar.parameters():
    param.requires_grad = True

# Also decrease the learning rate. Why we are doing this?
learning_rate = 0.001
optimizer = torch.optim.SGD(model_cifar.parameters(), lr=learning_rate, momentum=momentum)

In [49]:
# Finally do the same training we did before
best_model = train_and_validate(
    model=model_cifar, 
    train_loader=trainloader, 
    val_loader=testloader, 
    num_epoch=num_epoch, 
    criterion=criterion, 
    optimizer=optimizer, 
    device=device, 
    max_iter=100
)

evaluation: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 313/313 [00:19<00:00, 16.34it/s]


Epoch [1/5], train loss: 0.0488, train acc: 0.0491, val loss: 0.6007, val acc: 0.7954


evaluation: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 313/313 [00:19<00:00, 16.33it/s]


Epoch [2/5], train loss: 0.0323, train acc: 0.0546, val loss: 0.4479, val acc: 0.8480


evaluation: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 313/313 [00:19<00:00, 15.90it/s]


Epoch [3/5], train loss: 0.0284, train acc: 0.0555, val loss: 0.3915, val acc: 0.8634


evaluation: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 313/313 [00:19<00:00, 16.41it/s]


Epoch [4/5], train loss: 0.0261, train acc: 0.0569, val loss: 0.3608, val acc: 0.8750


evaluation: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 313/313 [00:19<00:00, 16.32it/s]

Epoch [5/5], train loss: 0.0211, train acc: 0.0583, val loss: 0.3363, val acc: 0.8864


In [50]:
# As the same type of analysis of predictations
predictations, true_labels = predict(model=best_model, data_loader=testloader, device=device)
print(classification_report(true_labels, predictations, target_names=cifar10_classes_list))

              precision    recall  f1-score   support

    airplane       0.90      0.88      0.89      1000
  automobile       0.95      0.94      0.95      1000
        bird       0.88      0.84      0.86      1000
         cat       0.83      0.72      0.77      1000
        deer       0.85      0.90      0.87      1000
         dog       0.79      0.88      0.83      1000
        frog       0.91      0.93      0.92      1000
       horse       0.90      0.91      0.90      1000
        ship       0.93      0.93      0.93      1000
       truck       0.94      0.93      0.93      1000

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



### DTD

<b><font color="red">[TODO]</font></b>: Conduct fine-tuning experiments for DTD dataset. What is the accuracy, how does it compare to the 'from-scratch' experiments?

### COCO-O

<b><font color="red">[TODO]</font></b>: Conduct fine-tuning experiments for COCO-O dataset. What is the accuracy, how does it compare to the 'from-scratch' experiments?

## Analysis of the results with FiftyOne lib

<b><font color="red">[TODO]</font></b>: Using the example from the previous 'Practice (Lecture 1)' session and the guidance from the provided [LINK](https://docs.voxel51.com/recipes/adding_classifications.html), analyze the COCO-O results using the FiftyOne tool. Specifically, focus on examining instances where the predictions do not align with the ground truth labels.